In [1]:
# SOLUSDT 1-Hour Historical Data Downloader using requests library with tqdm progress bar
# This script uses standard libraries, requests, and tqdm for visual progress tracking

import requests
import pandas as pd
import numpy as np
import datetime
import time
import os
import json
import tqdm  # Use regular tqdm, not tqdm.notebook

# First, make sure required packages are installed
# If you get an error, run: pip install requests pandas tqdm

# Function to convert date strings to milliseconds timestamp
def date_to_milliseconds(date_str):
    """Convert date string to milliseconds"""
    epoch = datetime.datetime.utcfromtimestamp(0)
    d = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    return int((d - epoch).total_seconds() * 1000)

# Function to convert interval to milliseconds
def interval_to_milliseconds(interval):
    """Convert interval string to milliseconds"""
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60,
    }

    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

# Function to get klines from Binance API
def get_binance_klines(symbol, interval, start_ms, end_ms=None, limit=1000):
    """Get klines from Binance API"""
    # API endpoint
    url = 'https://api.binance.com/api/v3/klines'
    
    # Prepare parameters
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_ms,
        'limit': limit
    }
    
    # Add end time if provided
    if end_ms:
        params['endTime'] = end_ms
    
    # Make the request
    response = requests.get(url, params=params)
    
    # Check if request was successful
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return []

# Function to get all historical klines with tqdm progress bar
def get_all_binance(symbol, interval, start_date, end_date=None):
    """
    Get all historical klines from Binance for the specified symbol and interval
    with tqdm progress bar
    
    Parameters:
    - symbol: string, the trading pair
    - interval: string, the interval of kline, e.g., '1h'
    - start_date: string, start date in format 'YYYY-MM-DD'
    - end_date: string, end date in format 'YYYY-MM-DD', if None, use current date
    
    Returns:
    - DataFrame with historical data
    """
    
    # Convert date strings to milliseconds
    start_ms = date_to_milliseconds(start_date)
    
    # If end_date not provided, use current date
    if end_date:
        end_ms = date_to_milliseconds(end_date)
    else:
        end_ms = int(datetime.datetime.now().timestamp() * 1000)
    
    # The list to store all klines
    all_klines = []
    
    # Initial start
    curr_start = start_ms
    
    # Binance limit for each request is 1000 data points
    limit = 1000
    
    # Get interval in milliseconds for progress calculation
    interval_ms = interval_to_milliseconds(interval)
    
    # Calculate approximate number of iterations needed
    if interval_ms:
        total_iterations = (end_ms - start_ms) // (interval_ms * limit) + 1
    else:
        # Default to a reasonable number if we can't calculate
        total_iterations = 100
    
    # Create tqdm progress bar - using regular tqdm instead of tqdm.notebook
    with tqdm.tqdm(total=total_iterations, desc=f"Downloading {symbol} {interval} data") as pbar:
        while curr_start < end_ms:
            # Fetch klines
            klines = get_binance_klines(
                symbol=symbol,
                interval=interval,
                start_ms=curr_start,
                end_ms=end_ms,
                limit=limit
            )
            
            # If no data returned or empty list, break the loop
            if not klines:
                break
            
            # Append data to our list
            all_klines.extend(klines)
            
            # Update start to get the next batch
            last_time = klines[-1][0]
            curr_start = last_time + 1  # Add 1 millisecond to avoid duplicate
            
            # Sleep to avoid hitting rate limits
            time.sleep(1)
            
            # Update progress bar
            pbar.update(1)
            
            # Update progress bar description
            curr_date = datetime.datetime.fromtimestamp(curr_start/1000).strftime('%Y-%m-%d')
            pbar.set_description(f"Downloading {symbol} {interval} data (up to {curr_date}, records: {len(all_klines)})")
    
    # If no data was collected, return empty DataFrame
    if not all_klines:
        print(f"No data found for {symbol} with interval {interval} from {start_date}")
        return pd.DataFrame()
    
    # Create DataFrame
    df = pd.DataFrame(all_klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # Set timestamp as index
    df.set_index('timestamp', inplace=True)
    
    # Convert string values to float for numerical columns
    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 
                'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']:
        df[col] = df[col].astype(float)
    
    return df

In [5]:
# Define parameters
symbol = "BTCUSDT"
interval = "15m"
# Solana was listed on Binance around April 2020, but we can start earlier
# and the API will return data from when it was actually available
start_date = "2025-01-01"  

In [12]:
# Create directory if it doesn't exist
if not os.path.exists('crypto_data'):
    os.makedirs('crypto_data')

# Get all historical klines
print(f"Starting download of complete {interval} history for {symbol}...")
try:
    df = get_all_binance(symbol, interval, start_date)
    
    # Check if we got any data
    if not df.empty:
        # Save the data to CSV
        filename = f'crypto_data/{symbol}_{interval}_complete_history.csv'
        df.to_csv(filename)
        print(f"✅ Data successfully saved to {filename}")
        print(f"📊 Total records: {len(df)}")
        print(f"📅 Date range: {df.index.min()} to {df.index.max()}")
    else:
        print("❌ No data was downloaded. Please check the symbol and date range.")
    
except Exception as e:
    print(f"❌ An error occurred: {e}")
    print("💡 Try installing required packages with: pip install requests pandas tqdm")

Starting download of complete 1m history for SOLUSDT...


✅ Data successfully saved to crypto_data/SOLUSDT_1m_complete_history.csv
📊 Total records: 840876
📅 Date range: 2024-01-01 00:00:00 to 2025-08-06 22:35:00
